In [1]:
import pandas as pd
from datetime import datetime
import pytz
import geopandas as gpd
import folium
from shapely.geometry import mapping
import json

# VTEC dictionaries
VTEC_PHENOMENA = {
    "AF": "Ashfall", "AS": "Air Stagnation", "BH": "Beach Hazard", "BS": "Blowing Snow",
    "BW": "Brisk Wind", "BZ": "Blizzard", "CF": "Coastal Flood", "CW": "Cold Weather",
    "DF": "Debris Flow", "DS": "Dust Storm", "DU": "Blowing Dust", "EC": "Extreme Cold",
    "EH": "Excessive Heat", "EW": "Extreme Wind", "FA": "Flood", "FF": "Flash Flood",
    "FG": "Dense Fog", "FL": "Flood", "FR": "Frost", "FW": "Red Flag", "FZ": "Freeze",
    "UP": "Freezing Spray", "GL": "Gale", "HF": "Hurricane Force Wind", "HI": "Inland Hurricane",
    "HS": "Heavy Snow", "HT": "Heat", "HU": "Hurricane", "HW": "High Wind", "HY": "Hydrologic",
    "HZ": "Hard Freeze", "IP": "Sleet", "IS": "Ice Storm", "LB": "Lake Effect Snow and Blowing Snow",
    "LE": "Lake Effect Snow", "LO": "Low Water", "LS": "Lakeshore Flood", "LW": "Lake Wind",
    "MA": "Marine", "MF": "Marine Dense Fog", "MH": "Marine Ashfall", "MS": "Marine Dense Smoke",
    "RB": "Small Craft for Rough", "RP": "Rip Currents", "SB": "Snow and Blowing", "SC": "Small Craft",
    "SE": "Hazardous Seas", "SI": "Small Craft for Winds", "SM": "Dense Smoke", "SN": "Snow",
    "SQ": "Snow Squall", "SR": "Storm", "SS": "Storm Surge", "SU": "High Surf", "SV": "Severe Thunderstorm",
    "SW": "Small Craft for Hazardous Seas", "TI": "Inland Tropical Storm", "TO": "Tornado", "TR": "Tropical Storm",
    "TS": "Tsunami", "TY": "Typhoon", "WC": "Wind Chill", "WI": "Wind", "WS": "Winter Storm",
    "WW": "Winter Weather", "ZF": "Freezing Fog", "ZR": "Freezing Rain"
}

VTEC_SIGNIFICANCE = {
    "W": "Warning", "Y": "Advisory", "A": "Watch", "S": "Statement", "O": "Outlook", "N": "Synopsis", "F": "Forecast"
}

NWS_COLORS = {
    "AF.W": "#A9A9A9", "AF.Y": "#696969", "AS.O": "#808080", "AS.Y": "#808080", "BH.S": "#40E0D0",
    "BW.Y": "#D8BFD8", "BZ.A": "#ADFF2F", "BZ.W": "#FF4500", "CF.A": "#66CDAA", "CF.S": "#6B8E23",
    "CF.W": "#228B22", "CF.Y": "#7CFC00", "DS.W": "#FFE4C4", "DS.Y": "#BDB76B", "DU.W": "#FFE4C4",
    "DU.Y": "#BDB76B", "EC.A": "#0000FF", "EC.W": "#0000FF", "EH.A": "#800000", "EH.W": "#C71585",
    "EH.Y": "#800000", "EW.W": "#FF8C00", "FA.A": "#2E8B57", "FA.W": "#00FF00", "FA.Y": "#00FF7F",
    "FF.A": "#2E8B57", "FF.S": "#8B0000", "FF.W": "#8B0000", "FG.Y": "#708090", "FL.A": "#2E8B57",
    "FL.S": "#00FF00", "FL.W": "#00FF00", "FL.Y": "#00FF7F", "FR.Y": "#6495ED", "FW.A": "#FFDEAD",
    "FW.W": "#FF1493", "FZ.A": "#00FFFF", "FZ.W": "#483D8B", "GL.A": "#FFC0CB", "GL.W": "#DDA0DD",
    "HF.A": "#9932CC", "HF.W": "#CD5C5C", "HT.Y": "#FF7F50", "HU.A": "#FF00FF", "HU.S": "#FFE4B5",
    "HU.W": "#DC143C", "HW.A": "#B8860B", "HW.W": "#DAA520", "HY.Y": "#00FF7F", "HZ.A": "#4169E1",
    "HZ.W": "#9400D3", "IS.W": "#8B008B", "LE.A": "#87CEFA", "LE.W": "#008B8B", "LE.Y": "#48D1CC",
    "LO.Y": "#A52A2A", "LS.A": "#66CDAA", "LS.S": "#6B8E23", "LS.W": "#228B22", "LS.Y": "#7CFC00",
    "LW.Y": "#D2B48C", "MA.S": "#FFDAB9", "MA.W": "#DB7093", "MF.Y": "#708090", "MH.Y": "#696969",
    "MS.Y": "#F0E68C", "RB.Y": "#D8BFD8", "RP.S": "#40E0D0", "SC.Y": "#D8BFD8", "SE.A": "#483D8B",
    "SE.W": "#D8BFD8", "SI.Y": "#D8BFD8", "SM.Y": "#F0E68C", "SQ.W": "#C71585", "SR.A": "#FFE4B5",
    "SR.W": "#9400D3", "SS.A": "#DB7FF7", "SS.W": "#C0C0C0", "SU.W": "#228B22", "SU.Y": "#BA55D3",
    "SV.A": "#DB7093", "SV.W": "#FFA500", "SW.Y": "#D8BFD8", "TO.A": "#FFFF00", "TO.W": "#FF0000",
    "TR.A": "#F08080", "TR.S": "#FFE4B5", "TR.W": "#B22222", "TS.A": "#FF00FF", "TS.W": "#FD6347",
    "TS.Y": "#D2691E", "TY.A": "#FF00FF", "TY.W": "#DC143C", "UP.A": "#4682B4", "UP.W": "#8B008B",
    "UP.Y": "#8B008B", "WC.A": "#5F9EA0", "WC.W": "#B0C4DE", "WC.Y": "#AFEEEE", "WI.Y": "#D2B48C",
    "WS.A": "#4682B4", "WS.W": "#FF69B4", "WW.Y": "#7B68EE", "ZF.Y": "#008080", "ZR.Y": "#DA70D6"
}

# Load the NWS alerts shapefile
nws_shapefile_path = 'nws_data/wwa_202407010000_202407180400.shp'
nws_data = gpd.read_file(nws_shapefile_path)

# Load the New York State boundary shapefile
ny_boundary_shapefile_path = 'state/State.shp'
ny_boundary = gpd.read_file(ny_boundary_shapefile_path)

# Reproject to UTM zone 18N (EPSG:32618)
nws_data = nws_data.to_crs(epsg=32618)
ny_boundary = ny_boundary.to_crs(epsg=32618)

# Spatial join to find the intersection
intersection = gpd.overlay(nws_data, ny_boundary, how='intersection')

# Filter by minimum area within NY (adjust threshold as needed)
min_area_threshold = 1e6  # 1,000,000 square meters (1 square kilometer)
intersection['area'] = intersection.geometry.area
filtered_intersection = intersection[intersection['area'] >= min_area_threshold]

/tmp/ipykernel_1211843/3007190681.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Convert to DataFrame for further processing
filtered_data = pd.DataFrame(filtered_intersection.drop(columns='geometry'))

# Convert UTC to Eastern Time
filtered_data['ISSUED'] = pd.to_datetime(filtered_data['ISSUED'], format='%Y%m%d%H%M')
filtered_data['ISSUED_ET'] = filtered_data['ISSUED'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

# Extract the date part for easier filtering
filtered_data['ISSUED_DATE_ET'] = filtered_data['ISSUED_ET'].dt.date

# Remove duplicates to keep only the 'NEW' status
unique_data = filtered_data[filtered_data['STATUS'] == 'NEW']

# Translate PHENOM and SIG columns
unique_data['PHENOM'] = unique_data['PHENOM'].map(VTEC_PHENOMENA)
unique_data['SIG'] = unique_data['SIG'].map(VTEC_SIGNIFICANCE)

/tmp/ipykernel_1211843/4185153247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data['PHENOM'] = unique_data['PHENOM'].map(VTEC_PHENOMENA)
/tmp/ipykernel_1211843/4185153247.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data['SIG'] = unique_data['SIG'].map(VTEC_SIGNIFICANCE)


In [3]:
# Create a new column for the combined alert type
unique_data['ALERT_TYPE'] = unique_data['PHENOM'] + ' ' + unique_data['SIG']

# Group by ISSUED_ET and ALERT_TYPE, and keep the row with the maximum AREA_KM2 for each group
max_area_alerts = unique_data.loc[
    unique_data.groupby(['ISSUED_ET', 'ALERT_TYPE'])['AREA_KM2'].idxmax()
]

# Print the deduplicated alerts with issued times, area covered, and alert type
#print(max_area_alerts[['ISSUED_ET', 'PHENOM', 'SIG', 'AREA_KM2']])

# Function to generate the summary for a specific date
def generate_summary(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    day_data = max_area_alerts[max_area_alerts['ISSUED_DATE_ET'] == date]
    
    summary = day_data.groupby('ALERT_TYPE').size().reset_index(name='count')
    
    return summary

# Generate the summary for a specific date
date_str = '2024-07-17'
summary = generate_summary(date_str)

# Display the summary
print(summary)

                    ALERT_TYPE  count
0          Flash Flood Warning      1
1               Flood Advisory      5
2               Marine Warning      7
3  Severe Thunderstorm Warning     16


/tmp/ipykernel_1211843/2297064069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data['ALERT_TYPE'] = unique_data['PHENOM'] + ' ' + unique_data['SIG']


# Save the filtered warnings to a new shapefile
output_shapefile_path = 'nws_data/filtered_warnings.shp'
filtered_intersection.to_file(output_shapefile_path)
print(f"Filtered warnings saved to {output_shapefile_path}")
geojson_data = json.loads(filtered_intersection.to_json())

# Function to create a Folium map with polygons
def create_map(gdf, map_filename='nws_warnings_map.html'):
    # Initialize a Folium map centered around New York State
    ny_center = [43.0, -75.0]
    folium_map = folium.Map(location=ny_center, zoom_start=6)

    # Convert GeoDataFrame to GeoJSON format
    gdf_json = json.loads(gdf.to_json())

    # Add polygons to the map
    for feature in gdf_json['features']:
        # Ensure 'properties' exists in the feature
        if 'properties' not in feature:
            continue

        # Get color and opacity based on 'SIG' value
        phenom = feature['properties'].get('PHENOM', '')
        sig = feature['properties'].get('SIG', '')
        key = f"{phenom}.{sig}"
        color = NWS_COLORS.get(key, 'black')  # Default to black if key not found

        if sig == 'A':  # Watches
            opacity = 0.2
        elif sig == 'W':  # Warnings
            opacity = 0.7
        else:  # Other types (e.g., advisories)
            opacity = 0.5

        geo_json = folium.GeoJson(
            feature,
            style_function=lambda x, color=color, opacity=opacity: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': opacity
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['PHENOM', 'SIG', 'area', 'ISSUED'],
                aliases=['Phenomenon', 'Significance', 'Area (sq m)', 'Issued Time'],
                localize=True
            )
        )
        geo_json.add_to(folium_map)

    # Save the map as an HTML file
    folium_map.save(map_filename)
    print(f"Map saved as {map_filename}")

# Ensure the GeoDataFrame is in lat/lon for Folium
filtered_gdf = filtered_intersection.to_crs(epsg=4326)

# Use the deduplicated alerts (max_area_alerts) to create the map
max_area_alerts_gdf = intersection[intersection['ISSUED'].isin(max_area_alerts['ISSUED'])]

# Ensure the GeoDataFrame is in lat/lon for Folium
max_area_alerts_gdf = max_area_alerts_gdf.to_crs(epsg=4326)

# Create the map
create_map(max_area_alerts_gdf)